In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, timedelta

In [14]:
%run ../interesting_options.py
%run -i my_heatmap_plotter.py
# %run -i my_3dbar_plotter.py

In [15]:
# %reset_selective -f plotter
%who_ls

['add_z_score',
 'calculate_horizon_date',
 'calls1',
 'calls2',
 'calls3',
 'calls4',
 'calls5',
 'date',
 'datetime',
 'datetime_valid',
 'df',
 'df_to_3Dbars',
 'filter_by_date',
 'filter_by_z_score',
 'interesting_options',
 'max_interest_by',
 'mis_df',
 'my_3dbar_plotter',
 'my_heatmap_plotter',
 'np',
 'o_chains_to_3d_df',
 'parse_contractSymbol',
 'pd',
 'plotter',
 'plt',
 'timedelta']

In [33]:
# load calls option chains
calls1, calls2, calls3, calls4, calls5 = (
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls1.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls2.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls3.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls4.csv', index_col=0),
    pd.read_csv('~/Desktop/code/mamacita/algo2/csv/pltr/calls5.csv', index_col=0)
)

print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)
print('calls4', calls4.shape)
print('calls5', calls5.shape)

# filter for interesting options
calls1, calls2, calls3, calls4, calls5 = (
    interesting_options(calls1, time_horizon='max', z_score_let=1, rmold=True),
    interesting_options(calls2, time_horizon='max', z_score_let=1, rmold=True),
    interesting_options(calls3, time_horizon='max', z_score_let=1, rmold=True),
    interesting_options(calls4, time_horizon='max', z_score_let=1, rmold=True),
    interesting_options(calls5, time_horizon='max', z_score_let=1, rmold=True),

)
# calls1.loc[calls1['exp'] < date.today()]
print('calls1', calls1.shape)
print('calls2', calls2.shape)
print('calls3', calls3.shape)
print('calls4', calls4.shape)
print('calls5', calls5.shape)

calls1 (436, 20)
calls2 (387, 20)
calls3 (391, 20)
calls4 (389, 20)
calls5 (397, 20)
calls1 (33, 23)
calls2 (32, 23)
calls3 (37, 23)
calls4 (36, 23)
calls5 (47, 23)


In [5]:
# my_3dbar_plotter accepts a misdf
def o_chains_to_3d_df(o_chains, 
                      x='date_strike', 
                      y='columns', 
                      z='openInterest', 
                      colsasdays=True
                     ):
    """
    Takes a number of option chains (each == one timeperiod of observation, default = 1 day).
    Returns a df where: 
        x = index (expiration date for MIS, or contract symbol) - x-axis
        y = column (observation time period, day1, day2 etc) - y-axis
        z = whatever we are looking at (open interest for MIS) - z-axis
        
    By default we count number of o_chains and create colums for each o_chain: colsasdays=True
    """
    
    # TODO - else y is dodgy
    columns = np.arange(0, len(o_chains), 1) if colsasdays is True else y 
    
    # I need to combine all o_chains indexes to make enough room in 3D bar plot
    # slice each chain
    def slice_chains(chains, x, y, z):
        out = []
        for i, chain in enumerate(chains):
         #   print(chain['exp']) # omg fuck this!!! introducing duplicates!
            new_chain = pd.DataFrame(chain[z].values, columns=[i], index=chain['date_strike'])
            out.append(new_chain)
        return out
   
    sliced = slice_chains(o_chains, x, y, z)
    out_df = pd.concat(sliced, axis=1)
    
    return out_df
    

In [6]:
# now plot a 3D bar chart
# x = expirations
# y = days of observation
# z = open_interest
mis_df = o_chains_to_3d_df([calls5, calls4, calls3, calls2, calls1])
# print(mis_df.shape)
# mis_df.to_csv('~/Desktop/code/mamacita/algo2/csv/pltr/mis_df2.csv')
# mis_df.to_csv('D:\\code\\test\\csv\\pltr\\mis_df1.csv') # damn df_combiner botches up

In [9]:
mis_df.head()

,0,1,2,3,4
2021-01-29 30.0,23966,25091.0,16341.0,13497.0,NaN
2021-01-29 40.0,15517,NaN,NaN,NaN,4840
2021-02-05 30.0,7301,10253.0,8616.0,7827.0,7827
2021-02-05 36.0,6940,NaN,NaN,NaN,NaN
2021-02-12 20.0,5124,NaN,NaN,NaN,NaN


In [10]:
# df = mis_df.reset_index(drop=True)
# remove past expirations
# df.head()
# df = df.loc[:, df['index'] > datetime.today()]
df = mis_df.reset_index()
df = df.fillna(0)
df.shape # (22, 5)


(24, 6)

In [12]:
%matplotlib qt 
print('df[index]', len(df['index']))
my_3dbar_plotter(df, 
                 depth=1, 
                 labels=['date_strike', 'days', 'open_interest']
                )

df[index] 24
len xticks: 24


In [19]:
mis_df.head()

,0,1,2,3,4
2021-01-29 30.0,23966,25091.0,16341.0,13497.0,NaN
2021-01-29 40.0,15517,NaN,NaN,NaN,4840
2021-02-05 30.0,7301,10253.0,8616.0,7827.0,7827
2021-02-05 36.0,6940,NaN,NaN,NaN,NaN
2021-02-12 20.0,5124,NaN,NaN,NaN,NaN


In [34]:
# heatmap
mic_df = o_chains_to_3d_df([calls1, calls2, calls3, calls4, calls5])
mic_df.fillna(0, inplace=True)
mic_df.values
mic_df.index = ['day'+ d for d in mic_df.index]

In [35]:
my_heatmap_plotter(mis_df.columns, mis_df.index, mis_df.values)